In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
import os
from numba import cuda 
plt.style.use("science.mplstyle")

from helpers.synthesis import *


dpi = 800


In [2]:
results_dir = "/global/ml4hep/spss/rrmastandrea/synth_SM_AD/NF_results_wide/"

n_sig_inj = [0, 300, 500, 750, 1000, 1200, 1500, 2000, 2500, 3000]#, 3500, 4000, 4500, 5000]# 10000]
n_sig_inj_SR = [0, 226, 376, 565, 753, 904, 1130, 1506, 1883, 2260]



seeds = [x for x in range(20)]


num_bkg_events = 320000


# Load in


In [3]:
#x_axis = np.logspace(-8, 0, 400)
x_axis = np.linspace(1e-5, 1, 100)


keys = ["feta","cathode", "curtains", "salad", "combined", "full_sup"]




rejections_dict = {k:{n:[] for n in n_sig_inj} for k in keys}                
sics_dict = {k:{n:[] for n in n_sig_inj} for k in keys}
max_sics_dict = {k:{n:[] for n in n_sig_inj} for k in keys}





In [4]:

for n in n_sig_inj:
    
    for seed in seeds:
        
        for k in keys:
            
            if k == "full_sup":
                tpr, sic, rejection, maxsic = get_sic_rejection(k, seed, 0, results_dir, num_bkg_events = num_bkg_events)
            else: 
                tpr, sic, rejection, maxsic = get_sic_rejection(k, seed, n, results_dir, num_bkg_events = num_bkg_events)
                
            
            interp_sic = np.interp(x_axis, tpr, sic)
            interp_rej = np.interp(x_axis, tpr, rejection)
            rejections_dict[k][n].append(interp_rej)
            sics_dict[k][n].append(interp_sic)
            max_sics_dict[k][n].append(maxsic)
            




[21.43947761 21.49604615 21.69403605 ...  0.99998594  1.00003594
  1.        ]
[23.0233968  23.05168107 23.10824961 ...  1.00073526  1.0007853
  1.        ]
[25.93667673 25.96496101 26.02152955 ...  1.00000938  1.00005938
  1.        ]
[21.8354574  22.00516303 22.06173157 ...  1.00001407  1.00006407
  1.        ]
[22.09001584 22.37285856 22.4294271  ...  1.00001875  1.00006876
  1.        ]
[22.8254069  22.96682825 23.0233968  ...  1.00007033  1.00012033
  1.        ]
[23.07996534 23.13653388 23.16481815 ...  1.00020319  1.00020006
  1.        ]
[31.8763737  31.93294224 31.98951078 ...  0.99999844  1.00004844
  1.        ]
[22.627417   22.74055408 22.79712263 ...  1.00207362  1.00207048
  1.        ]
[19.03531455 19.23330445 19.28987299 ...  1.00000156  1.00005157
  1.        ]
[21.72232032 21.92031022 21.97687876 ...  1.00207048  1.00206733
  1.        ]
[22.09001584 22.14658439 22.28800574 ...  1.00000938  1.00005938
  1.        ]
[24.55074744 24.57903171 24.6921688  ...  1.00026419 

[23.07996534 23.13653388 23.16481815 ...  1.00020319  1.00020006
  1.        ]
[31.8763737  31.93294224 31.98951078 ...  0.99999844  1.00004844
  1.        ]
[22.627417   22.74055408 22.79712263 ...  1.00207362  1.00207048
  1.        ]
[19.03531455 19.23330445 19.28987299 ...  1.00000156  1.00005157
  1.        ]
[21.72232032 21.92031022 21.97687876 ...  1.00207048  1.00206733
  1.        ]
[22.09001584 22.14658439 22.28800574 ...  1.00000938  1.00005938
  1.        ]
[24.55074744 24.57903171 24.6921688  ...  1.00026419  1.00031421
  1.        ]
[28.87824094 28.90652521 28.96309376 ...  1.00012973  1.00017974
  1.        ]
[28.31255552 28.39740833 28.45397687 ...  1.00000781  1.00005782
  1.        ]
[14.96237949 15.07551657 15.13208512 ...  0.999975    1.000025
  1.        ]
[28.4256926  28.65196677 28.76510386 ...  0.999975    1.000025
  1.        ]
[18.55448194 18.72418757 18.78075611 ...  1.00003907  1.00008907
  1.        ]
[16.94227848 17.02713129 17.08369983 ...  1.00017661  1.

# Take the avg

In [5]:



med_rejections_dict = {k:{n:[] for n in n_sig_inj} for k in keys}
med_sics_dict = {k:{n:[] for n in n_sig_inj} for k in keys}
med_max_sics_dict = {k:[[],[],[]] for k in keys} # mean, lower_p, upper_p
    
   

for n in n_sig_inj:
    
    for k in keys:
        
        med_rejections_dict[k][n] = get_med_percentile(rejections_dict[k][n])
        med_sics_dict[k][n] = get_med_percentile(sics_dict[k][n])
        
                 
        med_max_sics_dict[k][0].append(np.median(max_sics_dict[k][n]))
        med_max_sics_dict[k][1].append(np.percentile(max_sics_dict[k][n], 16))
        med_max_sics_dict[k][2].append(np.percentile(max_sics_dict[k][n], 84))
                        
        



for k in keys:
    med_max_sics_dict[k][0] = np.array(med_max_sics_dict[k][0])
    med_max_sics_dict[k][1] = np.array(med_max_sics_dict[k][1])
    med_max_sics_dict[k][2] = np.array(med_max_sics_dict[k][2])

In [ ]:

colors_dict = {"full_sup":"#56B4E9",
               "salad": "#009E73",
               "feta": "#CC79A7",
               "cathode":"#D55E00",
               "curtains":"#E69F00",
                "combined": "#e41a1c"}\

labels_dict = {"full_sup":"Full. Sup. ",
               "salad": "SALAD",
               "feta": "FETA",
               "cathode":"CATHODE",
               "curtains":"CURTAINs",
                "combined": "Combined"}

hatches_dict = {"full_sup":"Full. Sup. ",
               "salad": "\\",
               "feta": "/",
               "cathode":"-",
               "curtains":"|",
                "combined": "++"}


l = 3
f_1 = 20
f_2 = 30
alpha = 0.4
num_bkg = 121339

In [ ]:


x_vals = [n / num_bkg for n in n_sig_inj_SR]


fig = plt.figure(figsize = (15, 10)) 
for k in keys:
    
   
    plt.plot(x_vals, med_max_sics_dict[k][0], color = colors_dict[k], linewidth = l, alpha = alpha + 0.5)

    #plt.fill_between(x_vals, mean_max_sics_dict[k][0] -mean_max_sics_dict[k][1],
    #                    mean_max_sics_dict[k][0] + mean_max_sics_dict[k][1],
     #                    alpha = alpha, color = colors_dict[k], linewidth = 0)
        
    if k == "combined":
        plt.fill_between(x_vals, med_max_sics_dict[k][1],
                            med_max_sics_dict[k][2],
                             alpha = alpha+.3, facecolor = "none", edgecolor = colors_dict[k], 
                         linewidth = 0, hatch = hatches_dict[k], label = labels_dict[k])
    else:
        plt.fill_between(x_vals, med_max_sics_dict[k][1],
                            med_max_sics_dict[k][2],
                             alpha = alpha, color = colors_dict[k], 
                         linewidth = 0, label = labels_dict[k])
        

ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(x_vals)
ax2.set_xticklabels(n_sig_inj, size=f_1, rotation = 45)
ax2.set_xlabel(r"$n_\mathrm{sig}$", fontsize = f_2, labelpad = 10)


ax1.set_xticks(x_vals)
ax1.set_xticklabels([f"{np.round(100*x, 2)}\%" for x in x_vals], size=f_1, rotation = 45)
ax1.set_xlabel("S/B", fontsize = f_2)

"""
handles, labels = ax1.get_legend_handles_labels()
order = [0, 4, 1, 2, 3]
ax1.legend([handles[idx] for idx in order],[labels[idx] for idx in order],fontsize = f_1, loc = "lower right")
"""
ax1.legend(fontsize = f_1, loc = "lower right")

ax1.set_ylabel("max(SIC)", fontsize = f_2)
ax1.tick_params(axis='both', which='major', labelsize=f_1)
fig.savefig(f"plots/maxsic.pdf", dpi = dpi)

plt.show()





In [ ]:
for i, n in enumerate(n_sig_inj_SR):
    
    significance = n / np.sqrt(num_bkg)
    
    print(n, med_max_sics_dict["feta"][0][i]*significance)
    

# SIC

In [ ]:
n_to_plot = [750]

for n in n_to_plot:

    fig = plt.figure(figsize = (15, 10))

    for k in keys:
        
        
        plt.plot(x_axis, med_sics_dict[k][n][0], color = colors_dict[k], linewidth = l, alpha = alpha + 0.4)

   
        
        if k == "combined":
            plt.fill_between(x_axis, med_sics_dict[k][n][1],
                            med_sics_dict[k][n][2],
                             alpha = alpha+.3, facecolor = "none", edgecolor = colors_dict[k], 
                         linewidth = 0, hatch = hatches_dict[k], label = labels_dict[k])
        else:
                plt.fill_between(x_axis, med_sics_dict[k][n][1],
                            med_sics_dict[k][n][2],
                             alpha = alpha, color = colors_dict[k], 
                         linewidth = 0, label = labels_dict[k])
  

    plt.plot(x_axis, x_axis/np.sqrt(x_axis), label = "Random", color = "black", linewidth = l+1)
    
    ind = n_sig_inj.index(n)
    
    plt.text(0.5, 16, f"$n_S = {n}$\n$S/B = {np.round(100*n_sig_inj_SR[ind]/num_bkg, 2)}\%$", fontsize = f_2)

    """
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0, 4, 1, 2, 3]
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],fontsize = f_1)
    """
    
    plt.legend(fontsize = f_1)
    
    plt.xlabel("Signal Efficiency (True Positive Rate)", fontsize = f_2)    
    plt.ylabel("Significance Improvement Characteristic", fontsize = f_2)
    plt.tick_params(axis='both', which='major', labelsize=f_1)
    
    fig.savefig(f"plots/sic_{n}.pdf", dpi = dpi)
    
    plt.show()

# Rejection

In [ ]:
for n in n_to_plot:

    fig = plt.figure(figsize = (15, 10)) 
    for k in keys:
        
      
        plt.plot(x_axis, med_rejections_dict[k][n][0], color = colors_dict[k], linewidth = l, alpha = alpha + 0.4)

        if k == "combined":
            plt.fill_between(x_axis, med_rejections_dict[k][n][1],
                            med_rejections_dict[k][n][2],
                             alpha = alpha+.3, facecolor = "none", edgecolor = colors_dict[k], 
                         linewidth = 0, hatch = hatches_dict[k], label = labels_dict[k])
        else:
                plt.fill_between(x_axis, med_rejections_dict[k][n][1],
                            med_rejections_dict[k][n][2],
                             alpha = alpha, color = colors_dict[k], 
                         linewidth = 0, label = labels_dict[k])

    plt.plot(x_axis, 1.0/x_axis, label = "Random", color = "black", linewidth = l+1)
    
    ind = n_sig_inj.index(n)
    plt.text(0.5, 4000, f"$n_S = {n}$\n$S/B = {np.round(100*n_sig_inj_SR[ind]/num_bkg, 2)}\%$", fontsize = f_2)

    """
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0, 4, 1, 2, 3]
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],fontsize = f_1)  
    """   
    plt.legend(fontsize = f_1)  

    
    plt.ylim(5e-1, 5e4)
    plt.yscale("log")
    plt.xlabel("Signal Efficiency (True Positive Rate)", fontsize = f_2)  
    plt.ylabel("Rejection (1 / False Positive Rate)", fontsize = f_2)  
    #plt.title(f"Num injected events: {n}")
    plt.tick_params(axis='both', which='major', labelsize=f_1)
    
    fig.savefig(f"plots/rej_{n}.pdf", dpi = dpi)

    plt.show()

## SIC vs rej

In [ ]:
for n in n_to_plot:

    fig = plt.figure(figsize = (15, 10)) 
    for k in keys:
        

        plt.plot(med_rejections_dict[k][n][0], med_sics_dict[k][n][0], color = colors_dict[k], linewidth = l, alpha = alpha + 0.4)

        if k == "combined":
            plt.fill_between(med_rejections_dict[k][n][0], med_sics_dict[k][n][1],
                            med_sics_dict[k][n][2],
                             alpha = alpha+.3, facecolor = "none", edgecolor = colors_dict[k], 
                         linewidth = 0, hatch = hatches_dict[k], label = labels_dict[k])
        else:
                plt.fill_between(med_rejections_dict[k][n][0], med_sics_dict[k][n][1],
                            med_sics_dict[k][n][2],
                             alpha = alpha, color = colors_dict[k], 
                         linewidth = 0, label = labels_dict[k])

    plt.plot(1.0/x_axis[184:], x_axis[184:]/np.sqrt(x_axis[184:]), label = "Random", color = "black", linewidth = l+1)
    
    
    ind = n_sig_inj.index(n)
    plt.text(40, 16, f"$n_S = {n}$\n$S/B = {np.round(100*n_sig_inj_SR[ind]/num_bkg, 2)}\%$", fontsize = f_2)
    
    """
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0, 4, 1, 2, 3]
    plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order],fontsize = f_1)
    """

    plt.legend(fontsize = f_1)
    plt.xscale("log")
    plt.xlabel("Rejection (1 / False Positive Rate)", fontsize = f_2)  
    plt.ylabel("Significance Improvement Characteristic", fontsize = f_2)
    plt.tick_params(axis='both', which='major', labelsize=f_1)
    fig.savefig(f"plots/sic_vs_rej_{n}.pdf", dpi = dpi)
    


    plt.show()

In [ ]:

\
